In [1]:
#导入各种需要的库
#导入各种需要的库
import geemap
import ee
Map = geemap.Map()
from IPython.display import Image
import PySimpleGUI as sg
from tqdm.notebook import tqdm
from time import sleep
import csv
import logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [*] %(processName)s %(message)s"
)
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [6]:
js_snippet = """

var split = 0.8; 
var trainingPartition = withRandom.filter(ee.Filter.lt('random', split));
var validationPartition = withRandom.filter(ee.Filter.gte('random', split));

// train the RF classification model
var classifier = ee.Classifier.smileRandomForest(100)
    .train({
      features: trainingPartition, 
      classProperty: 'crop', 
      inputProperties: bands
    });

// Classify the input imagery.
var classified = image_com.classify(classifier);
"""
geemap.js_snippet_to_py(
    js_snippet, add_new_cell=True, import_ee=True, import_geemap=True, show_map=True
)

In [ ]:
# Assign a common name to the sensor-specific bands.
LC8_BANDS = ['B2',   'B3',    'B4',  'B5',  'B6',    'B7']; #Landsat 8
S2_BANDS  = ['B2',   'B3',    'B4',  'B8',  'B11',   'B12']; # Sentinel-2
STD_NAMES = ['blue', 'green', 'red', 'nir', 'swir1', 'swir2']
# landsat 8
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
           .filterBounds(roi) \
           .filterDate('2021-04-01','2021-08-01') \
           .map(rmL8Cloud) \
           .select(LC8_BANDS, STD_NAMES)

S2Col = ee.ImageCollection('COPERNICUS/S2') \
                .filterDate('2021-04-01', '2021-08-01') \
                .filterBounds(roi) \
                .map(rmS2cloud) \
                .select(S2_BANDS,STD_NAMES)

merged_L8S2 = ee.ImageCollection(l8.merge(S2Col)) \
                    .sort("system:time_start")

# generate the monthly composite

def func_ash(n):
    start = ee.Date('2021-04-01').advance(n, 'month')
    end = start.advance(1, 'month')
    return merged_L8S2.filterDate(start, end).median().clip(roi)

monthcomposite = ee.List.sequence(0, 1*2).map(func_ash).flatten()


In [ ]:


def func_euz(img)
    return img.clip(roi)

def mosaicByDate(imcol):
  # convert the ImageCollection into List
    imlist = imcol.toList(imcol.size())

    # Obtain the distinct image dates from the ImageCollection

    def func_kef(im):
        return ee.Image(im).date().format("YYYY-MM-dd")

    unique_dates = imlist.map(func_kef).distinct()   #删除重复的影像


  # mosaic the images acquired on the same date

    def func_wme(d):
        d = ee.Date(d)
        im = imcol.filterDate(d, d.advance(1, "day")).mosaic()  
        return im.set("system:time_start", d.millis(),"system:id", d.format("YYYY-MM-dd"))

    
    mosaic_imlist = unique_dates.map(func_wme)

    return ee.ImageCollection(mosaic_imlist)
# Get the VH collection.
collectionVH =  ee.ImageCollection('COPERNICUS/S1_GRD') \
                      .filter(ee.Filter.eq('instrumentMode', 'IW')) \
                      .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
                      .select(['VH']) \
                      .filterBounds(roi) \
                      .filterDate('2021-04-01', '2021-08-01')

collectionVH = collectionVH.map(func_euz)

collectionVH_Mosaiced = mosaicByDate(collectionVH)



In [5]:
#Defines a base date/time for the following examples.
BASE_DATE = ee.Date('2020-7-1T13:00', 'UTC')
print(BASE_DATE.getInfo(), 'The base date/time')

# Demonstrates basic usage.
print(BASE_DATE.advance(1, 'week').getInfo(), '+1 week')
print(BASE_DATE.advance(2, 'years').getInfo(), '+2 years')

#Demonstrates that negative delta moves back in time.
print(BASE_DATE.advance(-1, 'second').getInfo(), '-1 second')

{'type': 'Date', 'value': 1593608400000} The base date/time
{'type': 'Date', 'value': 1594213200000} +1 week
{'type': 'Date', 'value': 1656680400000} +2 years
{'type': 'Date', 'value': 1593608399000} -1 second


In [ ]:


split = 0.8
trainingPartition = withRandom.filter(ee.Filter.lt('random', split))
validationPartition = withRandom.filter(ee.Filter.gte('random', split))


classifier = ee.Classifier.smileRandomForest(100) \
    .train({
      'features': trainingPartition,
      'classProperty': 'crop',
      'inputProperties': bands
    })

# Classify the input imagery.
classified = image_com.classify(classifier)